### VQA-X

In [ ]:
import json

imgid2name = json.load(open('nle_data/coco_id2name.json', 'r'))

ques_list = json.load(open('nle_data/v2_OpenEnded_mscoco_train2014_questions.json', 'r'))['questions'] + \
            json.load(open('nle_data/v2_OpenEnded_mscoco_val2014_questions.json', 'r'))['questions']

ans_list  = json.load(open('nle_data/v2_mscoco_train2014_annotations.json', 'r'))['annotations'] + \
            json.load(open('nle_data/v2_mscoco_val2014_annotations.json', 'r'))['annotations']

exp_train = json.load(open('nle_data/VQA-X/textual/train_exp_anno.json', 'r'))
id2question = {str(q['question_id']): q['question'] for q in ques_list}
id2data = {}

for item in ans_list:
    
    qid = str(item['question_id'])
    
    if str(qid) not in exp_train.keys():
        continue
    
    id2data[qid] = {}
    id2data[qid]['question'] = id2question[qid]
    id2data[qid]['answers'] = item['answers']
    id2data[qid]['image_id'] = str(item['image_id'])
    id2data[qid]['image_name'] = imgid2name[str(item['image_id'])]
    exps = exp_train[str(qid)]
    id2data[qid]['explanation'] = [exps[i].lower().replace(".", "") for i in range(len(exps))]
    
with open('nle_data/VQA-X/vqaX_train.json', 'w') as w:
    json.dump(id2data, w)

### ACT-X

In [ ]:
import json
exp_train = json.load(open('nle_data/ACT-X/textual/exp_train_split.json'))
id2data = {}

for key,value in exp_train.items():

    id2data[key] = {}
    id2data[key]['answers'] = value['ans'].split(",")[0]
    id2data[key]['image_id'] = value['iid']
    id2data[key]['image_name'] = value['iid'] + '.jpg'
    id2data[key]['explanation'] = [value['exp'][i].lower().replace(".", "") for i in range(len(value['exp']))]
    
with open('nle_data/ACT-X/actX_train.json', 'w') as w:
    json.dump(id2data, w)

### eSNLI-VE

In [ ]:
import pandas as pd 
import json

exp_train = pd.read_csv('nle_data/eSNLI-VE/esnlive_train.csv').to_dict('records')

id2data = {}
for i,item in enumerate(exp_train):
    
    key = item['Unnamed: 0']
    id2data[key] = {}
    id2data[key]['hypothesis'] = item['hypothesis']
    id2data[key]['answers'] = item['gold_label']
    id2data[key]['image_name'] = item['Flickr30kID'] 
    id2data[key]['explanation'] = item['explanation'].lower().replace(".", "")
    
with open('nle_data/eSNLI-VE/esnlive_train.json', 'w') as w:
    json.dump(id2data, w)

### VCR

In [ ]:
import jsonlines as jsnl
import json

annotFolder = 'annotVCR/'
images_folder = 'images/vcr1images/'

def replace(x):
    if type(x) == list:
        names = " and ".join([objects[x[i]] + str(x[i]) for i in range(len(x))])
        return names
    return x


train_ann = jsnl.Reader(open(annotFolder + 'train.jsonl'))
data = [t for t in train_ann.iter()]
splits = json.load(open(annotFolder + 'vcr_splits.json', 'r'))
current_split = splits['train']
id2data = {}

for item in data:
         
    annID = item['annot_id']

    if annID not in current_split:
        continue
    
    ID = annID.split("-")[-1]
    metd = json.load(open(str(images_folder + item['metadata_fn'])))
    img_name = item['img_fn']
    objects = item['objects']
    question = item['question'][:-1]
    answer = item['answer_choices'][item['answer_label']][:-1]
    explanation = item['rationale_choices'][item['rationale_label']][:-1]
    boxes_data = metd['boxes']
    image_w, image_h = metd['width'], metd['height']

    str_question = " ".join(list(map(replace, question))).lower()
    str_answer =  " ".join(list(map(replace, answer))).lower()
    str_explanation =  " ".join(list(map(replace, explanation))).lower()
    
    id2data[ID] = {}
    id2data[ID]['question'] = str_question
    id2data[ID]['answers'] = str_answer
    id2data[ID]['explanation'] = str_explanation
    id2data[ID]['annot_id'] = annID
    id2data[ID]['img_name'] = img_name
    id2data[ID]['boxes_data'] = boxes_data
    id2data[ID]['image_w'] = image_w
    id2data[ID]['image_h'] = image_h
    id2data[ID]['objects'] = objects
    
assert len(id2data) == len(current_split)
with open('vcr_train.json', 'w') as w:
    json.dump(id2data, w)